In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/barCharts.ipynb
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

***vis_correct_incorrect_question* function**:
* Input: players_info dictionary
* Displays bar chart showing the total number of correct and incorrect alternatives selected by players in each multiple-choice question

In [1]:
outputResPerAlt=widgets.Output()
def vis_correct_incorrect_question():
    out=interactive(correct_incorrect_question,
            #constructing a dropdown to select how many data we want to see in visualisations for x axis
            xaxis=widgets.Dropdown(options=[('All datas',0),('5 per visualisation',5),('25 per visualisation',25),('50 per visualisation',50),('10 first',-1),('10 last',-2)],
                                description='x axe',
                                disabled=False),
             #constructing a dropdown to order for alphabetic keys or for values
            order=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
                                description='Order',
                                disabled=False),
             percentagePerAlternative=False,
             orderLabelValues=[("Order by all true and false responses values",0),
                               ("Order by true responses values","true"),
                               ("Order by false responses values","false")])
    md_text=widgets.HTML("<h4>Bar chart with number of correct and incorrect answers per question (alternative)</h4>")
    with outputResPerAlt:
        print("Updating visualisation with the current selection of players and alternatives...")
    return widgets.VBox([md_text,out, outputResPerAlt])

def correct_incorrect_question(xaxis, order,percentagePerAlternative, orderLabelValues):
    #get all players selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    alternatives_list_selected=checkboxesAlternativeSelected.get_selected_options()
    # alternatives correct and incorrect responses
    true_false_per_alternative = {}
    alt_max=0
    if len(alternatives_list_selected)>0:
        for alt in alternatives_list_selected:
            true_false_per_alternative[alt] = {}
            alt_true = 0
            alt_false = 0
            for player in playersSelected:
                if alt in players_info[player]["alternatives"].keys():
                    _, results = zip(*players_info[player]["alternatives"][alt])
                    alt_true += results.count("True") +results.count(True)
                    alt_false += results.count("False") +results.count(False)
                    if alt_true+alt_false>alt_max:
                        alt_max=alt_true+alt_false
            true_false_per_alternative[alt]["true"]=alt_true
            true_false_per_alternative[alt]["false"]=alt_false
        alt_max+=0.5
    if orderLabelValues==0:
        lambdafunc=lambda item:item[1]["true"]+item[1]["false"]
    else:
        lambdafunc=lambda item:item[1][orderLabelValues]
    true_false_per_alternative = orderDict(true_false_per_alternative,order,lambdafunc)
    nbgraphshown, true_false_per_alternative,N = organizeDictPerxAxisSelected(playersSelected,xaxis, true_false_per_alternative)
    for n in range(nbgraphshown):
        true_per_alternative=[true_false_per_alternative[n][alt]["true"] for alt in true_false_per_alternative[n].keys()]
        false_per_alternative=[true_false_per_alternative[n][alt]["false"] for alt in true_false_per_alternative[n].keys()]
        list_alt=[alt for alt in true_false_per_alternative[n].keys()]
        if n<nbgraphshown-1:
            ind = np.arange(N)
        else:
            ind = np.arange(len(list_alt))
        if percentagePerAlternative:
            if len(list_alt)>0:
                # Bar chart showing the total percentage of incorrect alternatives
                # selected by players in each multiple-choice question
                get_percentage_bar(true_per_alternative,false_per_alternative,list_alt)
            else:
                noDataToFillVis(10)
            ylabel="Percentage of good response"
            title="Percentage of good response per alternative"
        else:
            if len(list_alt)>0:
                # Bar chart showing the total number of correct and incorrect alternatives
                # selected by players in each multiple-choice question
                get_two_superposed_values_bars(ind,list_alt,
                               true_per_alternative, "Correct",'g',
                               false_per_alternative, "Incorrect",'r')
            else:
                noDataToFillVis(10)
            ylabel="Number of response"
            title="Number of correct and incorrect response per alternative"
        plt.xlabel("Alternative")
        plt.ylabel(ylabel)
        plt.title(title)
        plt.show()
    with outputResPerAlt:
        clear_output(wait=False)